<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Running on CPU

In [2]:
!pip install langchain langchain-core langchain-community --quiet
!pip install sentence_transformers FlagEmbedding chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.7 M

In [15]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
import chromadb
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
from tqdm import tqdm
import os

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)

In [13]:
def compute_approaches(company: str, distance: str, num: int):

  '''
  Retrive parent chunks using company name filter, distance/ similarity metric, number of chunks parameters
  '''

  # Get questions
  questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])

  # Create new column with the company name in the question
  questions['question_name'] = questions['question'].str.replace('company', company)

  # Initialize collection and ChromaDB vectorstore
  collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": distance})
  vectorstore = Chroma(client = persistent_client, collection_name="Reports", embedding_function=bge_embeddings,
                       persist_directory='/content/drive/MyDrive/Thesis/chromadb')
  vectorstore.persist()

  # Initialize a retriever
  parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
  child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
  big_chunks_retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store,
                                                 child_splitter=child_splitter, parent_splitter=parent_splitter,
                                                 search_kwargs={'filter': {'company': company}, 'k': num})

  # Invoke retriever without company name in the question
  results_list = []
  for i in tqdm(range(len(questions))):
    approach = f'approach_{company}_{distance}_1'
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        distance: response
    }))
  results = pd.concat(results_list, ignore_index=True)
  results.to_json(f'/content/drive/MyDrive/Thesis/retrievers/results_{approach}.json')

  # Invoke retriever with company name in the question
  results_list = []
  for i in tqdm(range(len(questions))):
    approach = f'approach_{company}_{distance}_2'
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        distance: response
    }))
  results = pd.concat(results_list, ignore_index=True)
  results.to_json(f'/content/drive/MyDrive/Thesis/retrievers/results_{approach}.json')

In [16]:
compute_approaches('COCA COLA CO', 'cosine', 2)

100%|██████████| 35/35 [01:53<00:00,  3.23s/it]


In [17]:
compute_approaches('COCA COLA CO', 'ip', 2)

100%|██████████| 35/35 [01:30<00:00,  2.58s/it]


In [18]:
compute_approaches('COCA COLA CO', 'l2', 2)

100%|██████████| 35/35 [01:47<00:00,  3.06s/it]


In [19]:
dst = ['cosine', 'ip', 'l2']
companies = ['AMAZON COM INC', 'PayPal Holdings, Inc.', 'GENERAL MILLS INC', 'Walmart Inc.']

for comp in companies:
  for d in dst:
    compute_approaches(comp, d, 2)

100%|██████████| 35/35 [00:59<00:00,  1.71s/it]


In [25]:
folder_path = '/content/drive/MyDrive/Thesis/retrievers/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [27]:
len(dataframes)

30